In [1]:
import os
import json
import shutil
import lzma

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

C:\Users\rdm42\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
download_dir = os.path.join('..','0_data','downloaded_posts')
metadata_paths = [os.path.join(download_dir,f) for f in os.listdir(download_dir) if f.endswith('.json.xz')]

additional_data = []

for i,path in enumerate(metadata_paths):

    with lzma.open(path, 'rt') as file:
        data_dict = json.load(file)
    
    short = data_dict['node']['shortcode']
    user  = data_dict['node']['owner']['username']
    likes = data_dict['node']['edge_media_preview_like']['count']
    caption = data_dict['node']['edge_media_to_caption']['edges'][0]['node']['text']
    long = os.path.split(path)[-1].split('.')[0]

    additional_data.append((short,user,likes,caption,long))

additional_data = pd.DataFrame(additional_data, columns=['Short','user','likes','caption','Long']) \
                    .drop(columns=['Long'])
additional_data['URL'] = additional_data.Short.apply(lambda code: f"https://www.instagram.com/p/{code}")
additional_data.head(3)

In [ ]:
with pd.ExcelFile('pictures_captions_autocaptions.xlsx') as xl:
    autocaptions = xl.parse()[['Short','Auto_caption']]
    
# autocaptions['Long'] = autocaptions.Long.str.extract("(.*_UTC)")[0]

In [ ]:
output_table = autocaptions.merge(additional_data, on='Short', how='left', validate='m:1') \
               [['Short','URL','user','likes','caption','Auto_caption']]

In [ ]:
output_table.to_clipboard(index=False)

In [ ]:
output_table[output_table.duplicated(keep=False)]

# Agregar obj_rate a LIWC

In [ ]:
import pandas as pd
import os

In [ ]:
with pd.ExcelFile("LIWC2015 Results (Ig posts sample - 11012024).xlsx") as xl:
    liwc_data = xl.parse()

In [ ]:
path = os.path.join('data','resources','database.xlsx')

with pd.ExcelFile(path) as xl:
    original_db = xl.parse(usecols='B:C').dropna()
    
original_db['Short'] = original_db.link.str.split('/').str[4]
original_db.drop_duplicates(subset=['Short'],inplace=True)

In [ ]:
df = liwc_data.merge(original_db.drop(columns=['link']).rename(columns={'Objectify (yes or no)':'objectification_flag'}),how='left',on='Short',validate='m:1')

cols = df.columns.tolist()
df[cols[:6] + [cols[-1]] + cols[6:-1]].to_excel('liwc_data_11012024.xlsx',index=False)

In [ ]:
# df.query("Short=='Cll7fIRsuo2'")

# Nueva data para LIWC de 1700 imagenes

In [2]:
def get_metadata(post_dir):
    metadata_path = os.path.join(post_dir,
                                 [file for file in os.listdir(post_dir)
                                  if file.endswith('.json.xz')][0])

    with lzma.open(metadata_path, 'rt') as file:
        data_dict = json.load(file)
        
    metadata = dict(
        user = data_dict['node']['owner']['username'],
        short = data_dict['node']['shortcode'],
        caption = data_dict['node']['edge_media_to_caption']['edges'][0]['node']['text']
    )
    
    return pd.Series(metadata)

In [3]:
path = os.path.join('..','0_data','2_data_women.xlsx')

with pd.ExcelFile(path) as xl:
    cat_women_imgs = xl.parse(index_col=0) \
                       .rename(columns={'Long':'img_path'}) \
                       .drop(columns=['Has_women'])
    
cat_women_imgs['post_dir'] = cat_women_imgs.img_path.apply(os.path.dirname)
cat_women_imgs['img_name'] = cat_women_imgs.img_path.apply(os.path.basename)

cat_women_imgs[['user','short','caption']] = cat_women_imgs.post_dir.apply(get_metadata)
cat_women_imgs['URL'] = cat_women_imgs.short.apply(lambda code: f"https://www.instagram.com/p/{code}")

cat_women_imgs = cat_women_imgs[['img_path','post_dir','img_name','short','user','URL','Auto_caption','caption']]

In [4]:
dir_path = os.path.join('samples for professors')

cat_women_imgs.drop(columns=['caption']).drop_duplicates() \
.to_excel(os.path.join(dir_path,'autocaptions_per_image.xlsx'),index=False)

cat_women_imgs.drop(columns=['Auto_caption','img_path','img_name']).drop_duplicates() \
.to_excel(os.path.join(dir_path,'original_captions_per_image.xlsx'),index=False)

<ipython-input-4-b1fa4cb8f58b>:3: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  cat_women_imgs.drop(columns=['caption']).drop_duplicates() \
<ipython-input-4-b1fa4cb8f58b>:6: UserWarning: Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.2.9' currently installed).
  cat_women_imgs.drop(columns=['Auto_caption','img_path','img_name']).drop_duplicates() \
